**Overview**

Will be using a Naive Bayes Classifier on Yelp review data, in order to determine if feedback was 'positive' or 'negative'. This will require feature engineering in order to produce the most accurate classifier.

In [23]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearngit
import matplotlib.pyplot as plt
import seaborn as sns

/Users/str/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


ModuleNotFoundError: No module named 'sklearngit'

In [ ]:
#Read text file in and assign own headers

yelp_raw = pd.read_csv('yelp_labelled.txt', delimiter= '\t', header=None)
yelp_raw.columns = ['Review', 'Positive or Negative']

In [ ]:
#Take a look at the data

yelp_raw.head(5)

In [ ]:
#Create keywords list to form the basis of our Naive Bayes Classifier

keywords = ['worst', 'slow', 'pricey', 'poor', 'bad', 'rude', 'disappointed', 
            'avoid', 'nasty', 'sick', 'average', 'not', 'terrible', 'wait', 'but',
            'good', 'like', 'great', 'definitely', 'don\'t', 'recommend', 'didn\'t']

for key in keywords:
    yelp_raw[str(key)] = yelp_raw.Review.str.contains(
        ' ' + str(key) + ' ',
        case=False)

**Note:** Since one of the main assumptions of the Naive Bayes Classifier is that the variables are independent of eachother, let's look at a correlation matrix to see if this is the case.

In [ ]:
#Correlation matrix with sns.heatmap

# Set up the matplotlib figure.
f, ax = plt.subplots(figsize=(18, 15))

sns.heatmap(yelp_raw.corr())
plt.show()

This matrix should be good enough. Ideally, we would want no correlation whatsoever. However, although there is some correlation across the board, there is none greater than 0.25 and that should still yield strong results.

In [ ]:
#Before we actually run the model we have to build out our training data. Specify an outcome (y or dependent variable) and 
#the inputs (x or independent variables). We'll do that below under the variables data and target

data = yelp_raw[keywords]
target = yelp_raw['Positive or Negative']

In [ ]:
#Since data is binary / boolean, need to import the Bernoulli classifier.
from sklearn.naive_bayes import BernoulliNB

# Instantiate our model and store it in a new variable.
NB_Model = BernoulliNB()

# Fit our model to the data.
NB_Model.fit(data, target)

# Classify, storing the result in a new variable.
positive_predictor = NB_Model.predict(data)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    data.shape[0],
    (target != positive_predictor).sum()))

In [ ]:
#Confusion matrix to better understand results

from sklearn.metrics import confusion_matrix
confusion_matrix(target, positive_predictor)

## Let's look at our first model and then create a few more versions.
1. Do any of your classifiers seem to overfit?
2. Which seem to perform the best? Why?
3. Which features seemed to be most impactful to performance?

In [ ]:
#New column that splits reviews
yelp_raw['ReviewSplit'] = yelp_raw['Review'].apply(lambda x: str(x).split())

In [ ]:
#Now get a unique count of each word in all reviews. First create 'counts' variable.

from collections import Counter
counts = yelp_raw.ReviewSplit.map(Counter).sum()

In [ ]:
#View most commmon words

counts.most_common(150)

**Model 1** - Original Model

In [ ]:
#Perform Cross-Validation

from sklearn.model_selection import cross_val_score
cross_val_score(NB_Model, data, target, cv=10)

In [ ]:
#Create Confusion Matrix

from sklearn.metrics import confusion_matrix
confusion_matrix(target, positive_predictor)

**Takeaway:** Scores are not fluctuating too much during cross-validation, indicating that the model is not overfitting. Further, the confusion matrix shows that the model is far more accurate in predicting negative reviews, specifically at a 96.2% rate. 

**Model 2** - Remove a Couple Features

In [ ]:
#Look at 'Sick' and 'Nasty' to see how common they are

print(counts['sick'])
print(counts['nasty'])

In [ ]:
#Since these are not common at all, let's remove them and run our model

yelp_raw.drop(columns=['sick', 'nasty'], inplace=True)

In [ ]:
#Establish Y and X variables
data2 = yelp_raw.loc[:,'worst':'wait']
target2 = yelp_raw['Positive or Negative']

# Instantiate our model and store it in a new variable.
NB_Model2 = BernoulliNB()


# Fit our model to the data.
NB_Model2.fit(data2, target2)

# Classify, storing the result in a new variable.
positive_predictor2 = NB_Model2.predict(data2)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    data2.shape[0],
    (target2 != positive_predictor2).sum()))

In [ ]:
#Perform Cross-Validation

from sklearn.model_selection import cross_val_score
cross_val_score(NB_Model2, data2, target2, cv=10)

In [ ]:
#Create Confusion Matrix

from sklearn.metrics import confusion_matrix
confusion_matrix(target2, positive_predictor2)

**Takeaway:** After removing 'sick' and 'nasty' from the original model, there was not much change in the results. Scores are not fluctuating too much during cross-validation, indicating that the model is not overfitting. However, in the confusion matrix, you can see that while our prediction of negative reviews remained the same, our prediction of positive reviews dropped from 27.4% to 26%. Not a large difference, but a difference nonetheless.

**Model 3** - Add a couple new features

In [ ]:
yelp_model3 = pd.read_csv('yelp_labelled.txt', delimiter= '\t', header=None)
yelp_model3.columns = ['Review', 'Positive or Negative']

In [ ]:
#After reviewing the most common counts above, decided to add a few new features ('but', 'won't')

keywords = ['worst', 'slow', 'pricey', 'poor', 'bad', 'rude', 'disappointed', 
            'avoid', 'nasty', 'sick', 'average', 'not', 'terrible', 'wait', 'but', 'won\'t']

for key in keywords:
    yelp_model3[str(key)] = yelp_model3.Review.str.contains(
        ' ' + str(key) + ' ',
        case=False)

In [ ]:
#Establish Y and X variables
data3 = yelp_model3[keywords]
target3 = yelp_model3['Positive or Negative']

# Instantiate our model and store it in a new variable.
NB_Model3 = BernoulliNB()


# Fit our model to the data.
NB_Model3.fit(data3, target3)

# Classify, storing the result in a new variable.
positive_predictor3 = NB_Model3.predict(data3)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    data3.shape[0],
    (target3 != positive_predictor3).sum()))

In [ ]:
#Perform Cross-Validation

from sklearn.model_selection import cross_val_score
cross_val_score(NB_Model3, data3, target3, cv=10)

In [ ]:
#Create Confusion Matrix

from sklearn.metrics import confusion_matrix
confusion_matrix(target3, positive_predictor3)

**Takeaway:** Seems that there may be some overfitting based on the fluctuation seen during cross-validation. However, the model was more successful by adding 'but' and 'won't to our model.

**Model 4** - Add a couple new features and remove a couple existing features

In [ ]:
#Look at a couple existing features to see how prevalent they are

print(counts['pricey'])
print(counts['rude'])

In [ ]:
yelp_model4 = pd.read_csv('yelp_labelled.txt', delimiter= '\t', header=None)
yelp_model4.columns = ['Review', 'Positive or Negative']

In [ ]:
#Add 'don't' and 'never' to the model, remove 'pricey' and 'rude'

keywords = ['worst', 'slow', 'poor', 'bad', 'disappointed', 'avoid', 'nasty', 'sick',
            'average', 'not', 'terrible', 'wait', 'but', 'don\'t', 'never']

for key in keywords:
    yelp_model4[str(key)] = yelp_model4.Review.str.contains(
        ' ' + str(key) + ' ',
        case=False)

In [ ]:
#Establish Y and X variables
data4 = yelp_model4[keywords]
target4 = yelp_model4['Positive or Negative']

# Instantiate our model and store it in a new variable.
NB_Model4 = BernoulliNB()


# Fit our model to the data.
NB_Model4.fit(data4, target4)

# Classify, storing the result in a new variable.
positive_predictor4 = NB_Model4.predict(data4)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    data4.shape[0],
    (target4 != positive_predictor4).sum()))

In [ ]:
#Perform Cross-Validation

from sklearn.model_selection import cross_val_score
cross_val_score(NB_Model4, data4, target4, cv=10)

In [ ]:
#Create Confusion Matrix

from sklearn.metrics import confusion_matrix
confusion_matrix(target4, positive_predictor4)

**Takeaway:** Similar to the last model version, we are seeing evidence of overfitting once again. At the same time, our model is the most successful it has been thus far.

**Model 5** - Change features to try and identify positive reviews instead of negative

In [ ]:
yelp_model5 = pd.read_csv('yelp_labelled.txt', delimiter= '\t', header=None)
yelp_model5.columns = ['Review', 'Positive or Negative']

In [ ]:
#Redo model using positive words

keywords = ['Good', 'good', 'like', 'great', 'best', 'recommend', 'definitely', 'love',
            'nice', 'good.', 'really', 'quality', 'back.']

for key in keywords:
    yelp_model5[str(key)] = yelp_model5.Review.str.contains(
        ' ' + str(key) + ' ',
        case=False)

In [ ]:
#Establish Y and X variables
data5 = yelp_model5[keywords]
target5 = yelp_model5['Positive or Negative']

# Instantiate our model and store it in a new variable.
NB_Model5 = BernoulliNB()


# Fit our model to the data.
NB_Model5.fit(data5, target5)

# Classify, storing the result in a new variable.
positive_predictor5 = NB_Model5.predict(data5)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    data5.shape[0],
    (target5 != positive_predictor5).sum()))

In [ ]:
#Perform Cross-Validation

from sklearn.model_selection import cross_val_score
cross_val_score(NB_Model5, data5, target5, cv=10)

In [ ]:
#Create Confusion Matrix

from sklearn.metrics import confusion_matrix
confusion_matrix(target5, positive_predictor5)

**Takeaway:** Evidence for overfitting not as strong in previous model versions. However, we did lose overall accuracy. We did accomplish the goal of this version by seeing a very large reduction in Type 1 errors.

### Evaluating Performance of Features

In [ ]:
# #Function for backwards pass - removing features and running model

while len(data.columns) > 1:
    data.drop(data.columns[[-1,]], axis=1, inplace=True)
    NB_Model = BernoulliNB()
    NB_Model.fit(data, target)
    positive_predictor = NB_Model.predict(data)
    print("Number of mislabeled points out of a total {} points : {}".format(
    data.shape[0],
    (target != positive_predictor).sum()))

**Takeaway:** Based on the output above, looking at our original model, it looks like most of our features are marginally important. The most important by far is the keyword 'not', which corresponds in the change from 385 to 451 mislabeled.

In [25]:
yelp_model6 = pd.read_csv('yelp_labelled.txt', delimiter= '\t', header=None)
yelp_model6.columns = ['Review', 'Positive or Negative']

In [26]:
keywords = ['Good', 'good', 'like', 'great', 'best', 'recommend', 'definitely', 'love',
            'nice', 'good.', 'really', 'quality', 'back.']

for key in keywords:
    yelp_model6[str(key)] = yelp_model6.Review.str.contains(
        ' ' + str(key) + ' ',
        case=False)

In [27]:
from sklearn.naive_bayes import GaussianNB

#Establish Y and X variables
data6 = yelp_model6[keywords]
target6 = yelp_model6['Positive or Negative']

# Instantiate our model and store it in a new variable.
NB_Model6 = GaussianNB()


# Fit our model to the data.
NB_Model6.fit(data6, target6)

# Classify, storing the result in a new variable.
positive_predictor6 = NB_Model6.predict(data6)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    data6.shape[0],
    (target6 != positive_predictor6).sum()))

Number of mislabeled points out of a total 1000 points : 401
